# Dependencies

In [24]:
from dotenv import load_dotenv
load_dotenv()

import os
import sys
import time

# CrewAI
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

from pydantic import BaseModel,Field


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")


In [25]:
import warnings
warnings.filterwarnings('ignore')

# Tools
Now, we will use the serper dev tool and scrape tool. The serper dev tool will search google for the given query and will fetch us the websites. 
After that, we will use the scrape tool to scrape the data from the websites. This way, we will be able to get the relevant context correctly 
from the websites.

In [26]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# Agents
Now, our task is to automate an event planning task, which can be done with the help of agents. 

In [27]:
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=dedent(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [28]:
logistics_manager = Agent(
    role='Logistics Manager',
    goal="Manage all logistics for the event including catering and equipment",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=dedent(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [29]:
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=dedent(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

# Pydantic Class
We will make our own custom class to hold information of all venues and events. The agents will be populating the data from the websites into this format.

In [30]:
class venueDetails(BaseModel):
    name : str = Field(...,description="This would be the name of the venue")
    address : str = Field(...,description="This would be the address of the venue")
    capacity : int = Field(...,description="This would be the capacity of the venue")
    booking_status : str = Field(...,description="This would be the booking status of the venue")

# Tasks
Now, we will be using some special settings/modules in our tasks, which are : 

* output_json : Here, we will tell the task that it has to answer based on a certain format, as defined above in the pydantic class
* output_file : Here, we will tell the task to write the output to a file.
* human_input=True : This will allow the task to take input from the user, which will be the query to search for the events.


In [40]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=venueDetails,
    output_file="venue_details.json",  
    agent=venue_coordinator
)

Interestingly enough, by setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [41]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date} located in {event_city}",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    output_file="logistics_details.md",
    agent=logistics_manager
)

In [42]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    output_file="marketing_report.md",
    agent=marketing_communications_agent
)

# Crew
Since we have set the `async_execution=True`, we can use the crew to run the tasks in parallel. The crew will be used to run the tasks in parallel i.e. now, the order does not matter and the tasks can run in parallel.

In [43]:
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

2025-02-24 16:36:57,584 - 8181330496 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Execution

In [44]:
event_details = {
    'event_topic': "AI Tech Innovation Conference - India",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders and software engineers "
                         "to explore future technologies in collaboration with the Indian government.",
    'event_city': "New Delhi",
    'tentative_date': "2025-03-15",
    'expected_participants': 500,
    'budget': 200000,
    'venue_type': "Conference Hall"
}

In [45]:
result = event_management_crew.kickoff(inputs=event_details)

# Agent: Venue Coordinator
## Task: Find a venue in New Delhi that meets criteria for AI Tech Innovation Conference - India.


# Agent: Venue Coordinator
## Thought: I need to search for venues in New Delhi that would be suitable for the AI Tech Innovation Conference, considering the specific requirements for the event.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"venues in New Delhi for AI Tech Innovation Conference\"}"
## Tool Output: 
{'searchParameters': {'q': 'venues in New Delhi for AI Tech Innovation Conference', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'AI Bharat Expo – AI International Exhibition, Expo and Conference in ...', 'link': 'https://www.aibharatexpo.com/', 'snippet': "The AI Bharat Expo will be co-located alongside Convergence India Expo which is India's largest technology showcase. As a whole, the expo will attract in excess ...", 'position': 1}, {'title': 'gaisa | Event in New Delhi - Townscript

In [47]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'Pragati Maidan, New Delhi, India',
 'booking_status': 'Available for booking',
 'capacity': 55000,
 'name': 'Bharat Mandapam'}


In [49]:
from IPython.display import Markdown
Markdown("logistics_details.md")

### Catering Services:
1. **Santosh Halwai & Caterers**
   - Link: [Justdial](https://www.justdial.com/Delhi/Caterers-For-Buffet-Upto-500-Persons/nct-11347883)
   - Specializes in catering for large events including buffet services.

2. **Best Catering Services in Delhi**
   - Link: [Sulekha](https://www.sulekha.com/catering-services/delhi)
   - Offers indoor & outdoor wedding catering services, cost estimates range from ₹450 per plate for vegetarian to ₹550 for non-vegetarian.

3. **Art Of Masala Foods Private Limited**
   - Link: [IndiaMART](https://www.indiamart.com/proddetail/party-catering-service-20795224633.html?srsltid=AfmBOoo8lv6EZmaqShEKuu7oB7_sT0-1LfN5VLK7ACspJcAntY0FiL4D)
   - Party catering service available at ₹500 per plate, accommodating up to 250 guests.

### Equipment Rental Services:
1. **E4 Events & Promotion**
   - Link: [IndiaMART](https://www.indiamart.com/proddetail/rental-services-5922231748.html?srsltid=AfmBOooe_JygUjWc-Q1vxkGpl_T498EHFJ2IhJMJE1MSzPlE1zoZgapV)
   - Offers various equipment rentals including DJ services tailored for events.

2. **Sulekha Event Rentals**
   - Link: [Sulekha](https://www.sulekha.com/event-equipment-supplies-hire/delhi)
   - Wide range of party equipment available for rent including tables, chairs, lighting, and audio-visual equipment.

3. **Gold Magic Events**
   - Link: [IndiaMART](https://www.indiamart.com/proddetail/video-conference-equipment-rental-service-21408976148.html?srsltid=AfmBOorMbM3cxZ-Gp6byXygZU5b2eCG0DRubkrPC9Pca70u7Bme-gEBO)
   - Provides video conference equipment rental services, suitable for larger groups.

4. **Audio Visual Equipment Rentals**
   - Link: [Travel Friend India](https://www.travelfriend.co.in/audio-visual-quoatation)
   - A comprehensive service for audio and visual equipment rental.

### Conclusion:
I have secured various catering options suitable for 500 participants in New Delhi as well as multiple equipment rental services to ensure all logistical aspects of the event on 2025-03-15 are accounted for. Please contact the providers directly to confirm arrangements and discuss specific needs for the event.

In [50]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Marketing Activities and Attendee Engagement for AI Tech Innovation Conference - India

## Event Overview
The AI Tech Innovation Conference is set to explore the latest advancements in artificial intelligence and technology, aiming to gather industry leaders and innovative minds in New Delhi on **March 15, 2025**. To ensure participation from at least 500 attendees, the following marketing activities and engagement strategies will be deployed:

## Marketing Strategies

### 1. **Utilize AI-Powered Marketing Tools**
Leverage AI-driven marketing platforms to analyze audience behavior, optimize campaigns, and personalize content. Tools which engage potential participants through targeted advertising can enhance reach and engagement.

### 2. **Email Marketing Campaigns**
- **Newsletter Series:** Launch a series of newsletters highlighting key speakers, topics, and early bird registration benefits.
- **Countdown Reminders:** Regular countdown emails leading up to the event create urgency and excitement.

### 3. **Social Media Engagement**
- **Dedicated Hashtag:** Create a unique hashtag for the conference to promote discussions across platforms.
- **Interactive Posts:** Polls, Q&As, and live streams featuring industry experts can foster engagement.
- **Influencer Partnerships:** Collaborate with industry influencers to extend reach and credibility.

### 4. **Networking Opportunities**
- **Panel Discussions:** Encourage discussions on trending topics in AI, allowing participants to engage with experts.
- **Breakout Sessions:** Organize smaller groups for in-depth conversations to facilitate networking.

### 5. **Incentives for Registration**
- **Early Bird Discounts:** Offer discounts for attendees who register early, enhancing motivation to secure a spot.
- **Group Packages:** Provide rates for organizations registering multiple individuals.

### 6. **Content Marketing**
- **Thought Leadership Articles:** Publish articles and blogs on the latest trends in AI to position the conference as a leading voice in the technology sector.
- **Webinars and Pre-Event Sessions:** Host pre-event webinars featuring keynote speakers to generate interest.

## Catering and Equipment Logistics
### Catering Options for 500 Participants
1. **Santosh Halwai & Caterers**
   - Specializes in buffet services for large events. [Justdial](https://www.justdial.com/Delhi/Caterers-For-Buffet-Upto-500-Persons/nct-11347883)

2. **Best Catering Services in Delhi**
   - Offers various catering services with estimates from ₹450 for vegetarian to ₹550 for non-vegetarian options. [Sulekha](https://www.sulekha.com/catering-services/delhi)

3. **Art Of Masala Foods**
   - Provides party catering at ₹500 per plate, suitable for 250 guests. [IndiaMART](https://www.indiamart.com/proddetail/party-catering-service-20795224633.html?srsltid=AfmBOoo8lv6EZmaqShEKuu7oB7_sT0-1LfN5VLK7ACspJcAntY0FiL4D)

### Equipment Rentals for the Event
1. **E4 Events & Promotion**
   - Offers comprehensive rental services, including DJ equipment. [IndiaMART](https://www.indiamart.com/proddetail/rental-services-5922231748.html?srsltid=AfmBOooe_JygUjWc-Q1vxkGpl_T498EHFJ2IhJMJE1MSzPlE1zoZgapV)

2. **Sulekha Event Rentals**
   - Provides a variety of party equipment including chairs, tables, and audio-visual gear. [Sulekha](https://www.sulekha.com/event-equipment-supplies-hire/delhi)

3. **Audio Visual Equipment Rentals**
   - Offers extensive services for audio and visual needs at events. [Travel Friend India](https://www.travelfriend.co.in/audio-visual-quoatation)

---

By implementing these strategies and securing necessary logistical services, the AI Tech Innovation Conference is positioned to attract a diverse array of attendees and foster meaningful connections across the industry.
```

# Conclusion
With this, we can see how we can create a team of agents which can work together to achieve a common goal. This acts an introduction to the concept of tools and how it can be used with agents to help agents give better outputs.